### Проверка заполненности данных плана именно тем, чем положено

In [1]:
%%time

# импорт библиотек
import pandas as pd
import numpy as np
import os
import warnings

warnings.simplefilter(action="ignore")

# загрузка исходного файла
df_ass = pd.read_csv(
    "C:/work/test_0/psy_05_07_2023_20_00.csv", sep=";", header=1, encoding="utf-8-sig"
)

# очистка полностю пустых строк
row_count_0 = df_ass.shape[0]
df_ass = df_ass.dropna(how='all')
print(f"Удалено {row_count_0 - df_ass.shape[0]} полностью пустых строк")

# заполнение пустых значений вниз
df_ass[df_ass.columns[0]] = df_ass[[df_ass.columns[0]]].ffill(axis=0)
df_ass[df_ass.columns[6]] = df_ass[
    [df_ass.columns[6]]
].ffill(axis=0)
df_ass[df_ass.columns[7]] = df_ass[
    [df_ass.columns[7]]
].ffill(axis=0)

# замена пропусков на 0
df_ass = df_ass.fillna(0)

# удаляем данные "по факту"
del_columns_list = []
for col in df_ass.columns:
    if col.split()[-1] == "факту":
        del_columns_list.append(col)
df_ass_plan = df_ass.drop(del_columns_list, axis=1)

# переименовываем столбцы, чтобы было только название оборудования
for col in df_ass_plan.columns[11:]:
    col_new = " ".join(col.split(" ")[1:-2])
    df_ass_plan.rename(columns={col: col_new}, inplace=True)

# удалим ненужные столбцы
del_columns_list = [1, 2, 4, 5, 6, 7, 8, 9]
df_ass_plan.drop(df_ass_plan.columns[del_columns_list], axis=1, inplace=True)

# вот с этим df_ass_plan дальше и работаем

df_ass_plan

Удалено 1 полностью пустых строк
CPU times: total: 188 ms
Wall time: 474 ms


,Регион,ИД,Название типа структурного подразделения или название типа медицинской организации,Весы,Малый хирургический набор,Молоток неврологический,Монитор пациента на 5 параметров,Набор реанимационный,Набор экстренной медицинской помощи содержащий лекарственные средства,Наборы для временной фиксации возбужденных больных,...,Централизованная подводка медицинских газов,Цифровой аналоговый рентгеновский аппарат,Экспресс измеритель концентрации глюкозы в крови портативный,Электрокардиограф многоканальный,Электрокоагулятор с функциями биполярной программируемой коагуляции,Электроэнцефалограф,Диктофон,Набор материалов для дефектологического исследования,Набор материалов для логопедического исследования,Набор материалов и оборудования для экспериментально психологического исследования
0,Алтайский край,46_1,Психиатрическая больница,35.0,0.0,79.0,0.0,19.0,19.0,13.0,...,1.0,2.0,17.0,7.0,0.0,4.0,3.0,3.0,1.0,134.0
1,Алтайский край,46_1_1,Отделение «Телефон доверия»,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Алтайский край,46_1_2,Кабинет медико-психологического консультирования,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Алтайский край,46_1_3,Психотерапевтический кабинет,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Алтайский край,46_1_4,Кабинет врача-психиатра,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5501,Ярославская область,57_14,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5502,Ярославская область,57_15,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5503,Ярославская область,57_16,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5504,Ярославская область,57_17,Кабинет врача-психиатра,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
# хорошая, годная функция
def compare(order_list, query_list):
    """вспомогательная функция для выявления различий между двумя списками"""
    order_list_result = order_list.copy()
    query_list_result = query_list.copy()
    for element_o in order_list:
        element_o_test = (
            element_o.lower().strip().replace(" ", "").replace(",", "").replace("-", "")
        )
        for element_q in query_list:
            element_q_test = (
                element_q.lower()
                .strip()
                .replace(" ", "")
                .replace(",", "")
                .replace("-", "")
            )
            if element_o_test == element_q_test:
                order_list_result.remove(element_o)
                query_list_result.remove(element_q)
            else:
                pass
    return order_list_result, query_list_result

In [4]:
# Психиатрическая больница
order_list_2 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для ультразвукового исследования",
    "Аппарат для электросудорожной терапии",
    "Аппарат искусственной вентиляции легких",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Дефибриллятор",
    "Компьютерный томограф",
    "Кресло-коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Магнитно-резонансный томограф",
    "Магнитный энцефалограф",
    "Молоток неврологический",
    "Набор реанимационный",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Пульсоксиметр",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Цифровой (аналоговый) рентгеновский аппарат",
    "Экспресс-измеритель концентрации глюкозы в крови портативный с набором тест-полосок",
    "Электрокардиограф многоканальный",
    "Электроэнцефалограф",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Кабинет врача-психиатра
order_list_0 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Общепсихиатрическое отделение
order_list_3 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Дефибриллятор",
    "Зонд для парентерального питания",
    "Кресло-коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Наборы для временной фиксации возбужденных больных",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс-измеритель концентрации глюкозы в крови портативный с набором тест-полосок",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Психотерапевтический кабинет
order_list_1 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения
order_list_5 = [
    "Тревожная кнопка",
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Общепсихиатрическое отделение для обслуживания детского населения
order_list_4 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья, психиатрической больницы)
order_list_6 = [
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
]

# Кабинет медико-психологического консультирования
order_list_7 = [
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Кушетка",
    "Диктофон",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Психоневрологический диспансер
order_list_8 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для ультразвукового исследования",
    "Аппарат искусственного дыхания портативный",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кресло-коляска",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Оборудование для проведения телемедицинских консультаций",
    "Ростомер",
    "Термометр",
    "Фонендоскоп",
    "Электроэнцефалограф",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Кабинет врача-психиатра детского
order_list_9 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Фонендоскоп",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Кабинет врача-сексолога
order_list_10 = [
    "Аппарат для вибростимуляции",
    "Аппарат для измерения артериального давления",
    "Аппарат для физиотерапии сексуальных дисфункций",
    "Аппарат локального отрицательного давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Гинекологическое кресло",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Тазомер",
    "Фаллоимитаторы (гинекологические расширители) разных размеров",
    "Фонендоскоп",
    "Электротермометр",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Кабинет (отделений) медико-психологической помощи
order_list_11 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Фонендоскоп",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Психиатрический дневной стационар
order_list_12 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Психиатрический дневной стационар для обслуживания детского населения
order_list_13 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кушетка",
    "Молоток неврологический",
    "Ростомер",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов для дефектологического исследования",
    "Набор материалов для логопедического исследования",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Отделение оказания психиатрической помощи при расстройствах психотического спектра в условиях дневного стационара
order_list_14 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Психиатрический дневной стационар для психосоциальной реабилитации
order_list_15 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Кровать медицинская общебольничная",
    "Молоток неврологический",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Отделение «Телефон доверия»
order_list_16 = [
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
]

# Отделения первого психотического эпизода
order_list_17 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Дефибриллятор",
    "Зонд для парентерального питания",
    "Кресло-коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Наборы для временной фиксации возбужденных больных",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс-измеритель концентрации глюкозы в крови портативный с набором тест-полосок",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]
# Психотерапевтическое отделение
order_list_18 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Дефибриллятор",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Пульсоксиметр",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Экспресс-измеритель концентрации глюкозы в крови портативный с набором тест-полосок",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Соматопсихиатрическое отделение
order_list_19 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Весы",
    "Дефибриллятор",
    "Кресло-коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Пульсоксиметр",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Экспресс-измеритель концентрации глюкозы в крови портативный с набором тест-полосок",
    "Электрокардиограф многоканальный",
]

# Отделений психосоциальной реабилитации
order_list_20 = [
    "Аппарат для измерения артериального давления",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Кресло-коляска",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
    "Электрокардиограф многоканальный",
    "Набор материалов и оборудования для экспериментально-психологического исследования",
]

# Отделение оказания психиатрической помощи в экстренной и неотложной формах
order_list_21 = [
    "Аппарат для измерения артериального давления",
    "Аппарат для измерения артериального давления неинвазивным способом",
    "Аппарат для проведения плазмафереза",
    "Аппарат искусственной вентиляции легких (CMV, SIMV, CPAP) с мониторированием дыхательного и минутного объема дыхания, давления в контуре аппарата",
    "Аппарат наркозный (полуоткрытый и полузакрытый контуры) с дыхательным автоматом, волюметром, монитором концентрации кислорода, углекислоты и герметичности дыхательного контура (не менее одного испарителя для испаряемых анестетиков)",
    "Аппарат электросудорожной терапии",
    "Аспиратор электрический",
    "Бактерицидный облучатель/очиститель воздуха/устройство для обеззараживания и (или) фильтрации воздуха и (или) дезинфекции поверхностей",
    "Дефибриллятор",
    "Дыхательный мешок для ручной искусственной вентиляции легких",
    "Кровать медицинская общебольничная",
    "Кушетка",
    "Молоток неврологический",
    "Монитор пациента на 5 параметров (оксиметрия, неинвазивное артериальное давление, электрокардиограмма, частота дыхания, температура)",
    "Набор для интубации трахеи, включая ларингеальную маску, ларингеальную маску для интубации трахеи и комбинированную трубку",
    "Набор экстренной медицинской помощи, содержащий лекарственные средства",
    "Насос инфузионный",
    "Насос шприцевой",
    "Сейф для хранения сильнодействующих и психотропных веществ",
    "Стойка для внутривенных инфузий",
    "Термометр",
    "Тумба прикроватная",
    "Фонендоскоп",
    "Холодильник медицинский",
]

# Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)
order_list_22 = []

In [5]:
# Словарь соответствия типа МО и списка девайсов
type_devices_dict = {
    "Кабинет врача-психиатра": order_list_0,
    "Психотерапевтический кабинет": order_list_1,
    "Психиатрическая больница": order_list_2,
    "Общепсихиатрическое отделение": order_list_3,
    "Общепсихиатрическое отделение для обслуживания детского населения": order_list_4,
    "Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения": order_list_5,
    "Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья, психиатрической больницы)": order_list_6,
    "Кабинет медико-психологического консультирования": order_list_7,
    "Психоневрологический диспансер": order_list_8,
    "Кабинет врача-психиатра детского": order_list_9,
    "Кабинет врача-сексолога": order_list_10,
    "Кабинет (отделение) медико-психологической помощи": order_list_11,
    "Психиатрический дневной стационар": order_list_12,
    "Психиатрический дневной стационар для обслуживания детского населения": order_list_13,
    "Отделение оказания психиатрической помощи при расстройствах психотического спектра в условиях дневного стационара": order_list_14,
    "Психиатрический дневной стационар для психосоциальной реабилитации": order_list_15,
    "Отделение «Телефон доверия»": order_list_16,
    "Отделение первого психотического эпизода": order_list_17,
    "Психотерапевтическое отделение": order_list_18,
    "Соматопсихиатрическое отделение": order_list_19,
    "Отделение психосоциальной реабилитации": order_list_20,
    "Отделение оказания психиатрической помощи в экстренной и неотложной формах": order_list_21,
    "Психоневрологический диспансер (центр психического здоровья, диспансерное отделение психиатрической больницы)": order_list_22,
}

In [6]:
# список типов МО
mo_type_list = (
    df_ass_plan[
        "Название типа структурного подразделения или название типа медицинской организации"
    ]
    .unique()
    .tolist()
)
mo_type_list

['Психиатрическая больница',
 'Отделение «Телефон доверия»',
 'Кабинет медико-психологического консультирования',
 'Психотерапевтический кабинет',
 'Кабинет врача-психиатра',
 'Кабинет активного диспансерного наблюдения и проведения амбулаторного принудительного лечения',
 'Психиатрический дневной стационар',
 'Психиатрический дневной стационар для психосоциальной реабилитации',
 'Общепсихиатрическое отделение',
 'Отделение первого психотического эпизода',
 'Отделение психосоциальной реабилитации',
 'Общепсихиатрическое отделение для обслуживания детского населения',
 'Отделение оказания психиатрической помощи при расстройствах психотического спектра в условиях дневного стационара',
 'Психиатрический дневной стационар для обслуживания детского населения',
 'Психотерапевтическое отделение',
 'Кабинет (отделение) медико-психологической помощи',
 'Кабинет врача-психиатра детского',
 'Организационно-методический отдел (кабинет) психоневрологического диспансера (центра психического здоровья

In [7]:
# общий спиисок результатаов
result_list = (
    []
)  # в таком случае получается список списков результатов по типу лечебницы. А мне надо почти гладкий список, где списки - только списки девайсов
mo_type_result_list = (
    []
)  # буду дополнять этот список дальше и дальше, при смене типа МО
# перебираем лечебницы по типу МО и создаём выборку из df_ass_plan, называемую df_test
for mo_type in mo_type_list:
    df_test = df_ass_plan[
        df_ass_plan[
            "Название типа структурного подразделения или название типа медицинской организации"
        ]
        == mo_type
    ]
    # внутри типа МО для каждой лечебницы формируем словарь заявленного оборудования
    device_dict = {}
    for index, row in df_test.iterrows():
        device_list = []
        key = row["ИД"]
        for col in df_test.columns[3:]:
            if row[col] > 0:
                device_list.append(col)
        device_dict[key] = device_list

    # пустой список для сохранения результата по типу лечебницы
    # mo_type_result_list = []

    # передаём в order_list список девайсов для текущего типа МО
    order_list = type_devices_dict[mo_type]

    # перебираем лечебницы внутри типа МО
    for key in device_dict.keys():  # key - индекс лечебницы внутри типа МО
        # для каждой лечебницы выбираем список запрошенного оборудования query_list
        query_list = device_dict[key]
        # сравниваем запрошенное с приказом
        order_list_result, query_list_result = compare(order_list, query_list)
        # складываем результаты в список результатов по типу лечебницы
        mo_type_result_list.append(mo_type)
        mo_type_result_list.append(key)
        mo_type_result_list.append(order_list_result)
        mo_type_result_list.append(query_list_result)
    # присоединяем список результатаов по типу лечебницы к общему списку разельтатов
    result_list.append(mo_type_result_list)  # создам, но работать с ним не буду пока

In [8]:
# преобразование словаря во фрейм данных
# нарезаем список на список списков по числу столобцов
mo_type_result_list_df = [
    mo_type_result_list[i : i + 4] for i in range(0, len(mo_type_result_list), 4)
]
# преобразуем словарь словарей в фрейм
df_mo_type_result = pd.DataFrame(
    mo_type_result_list_df, columns=["тип МО", "код МО", "no", "more"]
)

In [9]:
# прикрутим к этому всему регион
df_mo_type_result = df_mo_type_result.merge(
    df_ass_plan, left_on="код МО", right_on="ИД", how="left"
)
df_mo_type_result = df_mo_type_result[["Регион", "тип МО", "код МО", "no", "more"]]

In [21]:
df_mo_type_result

,Регион,тип МО,код МО,no,more
0,Алтайский край,Психиатрическая больница,46_1,[Бактерицидный облучатель/очиститель воздуха/у...,[Наборы для временной фиксации возбужденных бо...
1,Амурская область,Психиатрическая больница,1_1,"[Аппарат для измерения артериального давления,...",[]
2,Архангельская область,Психиатрическая больница,3_1,[Бактерицидный облучатель/очиститель воздуха/у...,[Наборы для временной фиксации возбужденных бо...
3,Астраханская область,Психиатрическая больница,4_1,"[Аппарат для электросудорожной терапии, Аппара...",[Наборы для временной фиксации возбужденных бо...
4,Белгородская область,Психиатрическая больница,68_29,"[Аппарат для измерения артериального давления,...",[]
...,...,...,...,...,...
5513,Республика Башкортостан,Отделение оказания психиатрической помощи в эк...,82_36_44,"[Аппарат для измерения артериального давления,...",[]
5514,Республика Дагестан,Отделение оказания психиатрической помощи в эк...,45_38_3,"[Аппарат для проведения плазмафереза, Аппарат ...","[Весы, Малый хирургический набор, Набор реаним..."
5515,Республика Татарстан,Отделение оказания психиатрической помощи в эк...,22_25_37,"[Аппарат для измерения артериального давления,...",[]
5516,Сахалинская область,Отделение оказания психиатрической помощи в эк...,40_26_1_1,"[Аппарат для измерения артериального давления,...",[]


In [10]:
# для выгрузки в файл
df_mo_type_result["забыли заказать"] = df_mo_type_result["no"].apply(
    lambda x: ", ".join(x)
)
df_mo_type_result["заказали сверх"] = df_mo_type_result["more"].apply(
    lambda x: ", ".join(x)
)
df_mo_type_result.drop(["no", "more"], axis=1, inplace=True)
df_mo_type_result.to_csv(
    "C:/work/test_0/result_0.csv", sep=";", encoding="utf-8-sig", index=False
)

In [23]:
df_mo_type_result

,Регион,тип МО,код МО,забыли заказать,заказали сверх
0,Алтайский край,Психиатрическая больница,46_1,Бактерицидный облучатель/очиститель воздуха/ус...,Наборы для временной фиксации возбужденных бол...
1,Амурская область,Психиатрическая больница,1_1,"Аппарат для измерения артериального давления, ...",
2,Архангельская область,Психиатрическая больница,3_1,Бактерицидный облучатель/очиститель воздуха/ус...,Наборы для временной фиксации возбужденных бол...
3,Астраханская область,Психиатрическая больница,4_1,"Аппарат для электросудорожной терапии, Аппарат...",Наборы для временной фиксации возбужденных бол...
4,Белгородская область,Психиатрическая больница,68_29,"Аппарат для измерения артериального давления, ...",
...,...,...,...,...,...
5513,Республика Башкортостан,Отделение оказания психиатрической помощи в эк...,82_36_44,"Аппарат для измерения артериального давления, ...",
5514,Республика Дагестан,Отделение оказания психиатрической помощи в эк...,45_38_3,"Аппарат для проведения плазмафереза, Аппарат и...","Весы, Малый хирургический набор, Набор реанима..."
5515,Республика Татарстан,Отделение оказания психиатрической помощи в эк...,22_25_37,"Аппарат для измерения артериального давления, ...",
5516,Сахалинская область,Отделение оказания психиатрической помощи в эк...,40_26_1_1,"Аппарат для измерения артериального давления, ...",


# остановились тут

In [56]:
df_ass_plan_group["Весы"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8 entries, 0 to 7
Series name: Весы
Non-Null Count  Dtype  
--------------  -----  
8 non-null      float64
dtypes: float64(1)
memory usage: 196.0 bytes


In [57]:
df_ass_plan_group.insert(2, "device", "")

In [58]:
device_dict = {}
for index, row in df_ass_plan_group.iterrows():
    device_list = []
    key = row[
        "Название типа структурного подразделения или название типа медицинской организации"
    ]
    for col in df_ass_plan_group.columns[3:]:
        if row[col] > 0:
            device_list.append(col)
    device_dict[key] = device_list

In [59]:
df_devices = pd.DataFrame(list(device_dict.items()), columns=["dept", "devices"])

In [60]:
df_devices

,dept,devices
0,Кабинет активного диспансерного наблюдения и п...,"[Весы, Молоток неврологический, Фонендоскоп, К..."
1,Кабинет врача-психиатра,"[Весы, Молоток неврологический, Ростомер, Фоне..."
2,Кабинет медико-психологического консультирования,"[Кушетка, Бактерицидный облучатель, Диктофон, ..."
3,Общепсихиатрическое отделение,"[Весы, Молоток неврологический, Набор экстренн..."
4,Общепсихиатрическое отделение для обслуживания...,"[Весы, Молоток неврологический, Набор реанимац..."
5,Организационно-методический отдел (кабинет) пс...,[Бактерицидный облучатель]
6,Психиатрическая больница,"[Весы, Малый хирургический набор, Молоток невр..."
7,Психотерапевтический кабинет,"[Весы, Молоток неврологический, Фонендоскоп, К..."


In [40]:
df_devices["dev"] = df_devices["devices"].apply(lambda x: ", ".join(x))
df_devices.drop("devices", axis=1, inplace=True)

In [42]:
df_devices

,dept,dev
0,Кабинет активного диспансерного наблюдения и п...,"Весы, Молоток неврологический, Фонендоскоп, Ку..."
1,Кабинет врача-психиатра,"Весы, Молоток неврологический, Ростомер, Фонен..."
2,Кабинет медико-психологического консультирования,"Кушетка, Бактерицидный облучатель, Диктофон, Н..."
3,Общепсихиатрическое отделение,"Весы, Молоток неврологический, Набор экстренно..."
4,Общепсихиатрическое отделение для обслуживания...,"Весы, Молоток неврологический, Набор реанимаци..."
5,Организационно-методический отдел (кабинет) пс...,Бактерицидный облучатель
6,Психиатрическая больница,"Весы, Малый хирургический набор, Молоток невро..."
7,Психотерапевтический кабинет,"Весы, Молоток неврологический, Фонендоскоп, Ку..."


In [61]:
df_devices.to_csv(
    "C:/work/test_0/result_0.csv", sep=";", encoding="utf-8-sig", index=False
)

In [166]:
order_result, query_result = compare(order_list, query_list)

In [167]:
order_result

['Тревожная кнопка', 'нож Для лоботомии']

In [168]:
query_result

['Пизда с ушами', 'моЁ второе Я']